In [62]:
%load_ext autoreload
%autoreload 2
from Methods import sentenceSimilarityMethods,JobOffers,dataframeHelpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import json
import os
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
from scipy import spatial

In [64]:
autoreload JobOffers,dataframeHelpers

In [70]:
# Only run this if you want to redo the translations
JobOffers.writeTranslatedJobs(100)

In [71]:
slimmedActsDf = dataframeHelpers.buildActivitiesDF()

In [72]:
slimmedActsDf

,ACTIVITY_ID,ACTIVITY_CONTENT
0,0,Where and how are the information and data gai...
1,1,"Observing, receiving, and otherwise obtaining..."
3,4.A.1.a.1.G01,Collect geographic information systems (GIS) d...
4,4.A.1.a.1.G02,Collect geological data.
5,4.A.1.a.1.G03,Collect data about green practices or programs.
...,...,...
225,4.A.4.b.6.G08,Provide technical guidance on green projects.
226,226,Performing day-to-day administrative tasks suc...
227,4.A.4.c.1.G01,Obtain permits.
228,228,"Recruiting, interviewing, selecting, hiring, ..."


In [75]:
#got stuck here
jobsDf = dataframeHelpers.buildJobsDF()

In [76]:
jobsDf

,JOB_ID,ISCO,JOB_CONTENT,JOB_SCORES
0,12220025,21420,[We are a successful and independent engineeri...,"((tf.Tensor(-0.04822776, shape=(), dtype=float..."
1,10814567,21420,[We are a prestigious and successful engineeri...,"((tf.Tensor(-0.020631889, shape=(), dtype=floa..."
2,12401674,21420,[Bill Weyermann Partner AG has been producing ...,"((tf.Tensor(0.055472188, shape=(), dtype=float..."
3,12473085,62100,[Administration Communale de Valderuzforstarti...,"((tf.Tensor(0.019481275, shape=(), dtype=float..."
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,"((tf.Tensor(-0.022287764, shape=(), dtype=floa..."
...,...,...,...,...
95,13469747,62100,[Commune de Lausanne Service of the Parcs Et D...,"((tf.Tensor(0.06673486, shape=(), dtype=float3..."
96,13494906,24210,[Quality Assurance Engineer Bern Switzerland Y...,"((tf.Tensor(-0.010205403, shape=(), dtype=floa..."
97,13517245,21420;31180,[Solution-oriented creative and dynamic We are...,"((tf.Tensor(-0.055482067, shape=(), dtype=floa..."
98,13461980,21620,[Green Blue Gardener Garden and Landscaping Ef...,"((tf.Tensor(-0.008430735, shape=(), dtype=floa..."


In [37]:
slimmedActsDf["ACTIVITY_SCORES"] = allEmbeddings[-1*(len(slimmedActsDf)):]

In [38]:
len(slimmedActsDf["ACTIVITY_SCORES"][0])

512

In [39]:
dist = spatial.distance.cosine(jobsDf["JOB_SCORES"][0][0], slimmedActsDf["ACTIVITY_SCORES"][0])

In [40]:
dist

0.9204563945531845

In [41]:
resultsDf = pd.DataFrame()

In [42]:
resultsDf["JOB_ID"] = jobsDf["JOB_ID"]

In [43]:
resultsDf["JOB_SCORES"] = jobsDf["JOB_SCORES"]

In [44]:
resultsDf = resultsDf.merge(slimmedActsDf, how = "cross")

In [45]:
len(resultsDf["ACTIVITY_SCORES"])

21900

In [46]:
resultsDf = resultsDf[["JOB_ID", "ACTIVITY_ID", "JOB_SCORES", "ACTIVITY_SCORES"]]


In [47]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3..."
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32..."
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3..."
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32..."
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3..."
...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float..."
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float..."
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3..."
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float..."


In [48]:
row = resultsDf.loc[0,:]

In [49]:
def compareSentences(row):
    for sentence in resultsDf["JOB_SCORES"]:
        return spatial.distance.cosine(row["JOB_SCORES"][sentence], row["ACTIVITY_SCORES"])

In [50]:
def distCalc(row):
    distArray = []
    for entry in row["JOB_SCORES"] :
        dist = spatial.distance.cosine(entry, row["ACTIVITY_SCORES"])
        distArray.append(dist)
    return distArray
    

In [51]:
row = resultsDf.loc[0,:]

In [52]:
jobsDfRow = jobsDf.loc[0,:]

In [53]:
resultsDf["DISTANCES"] = resultsDf.apply(distCalc, axis =1)

In [54]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896..."
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960..."
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3...","[1.02582386136055, 1.048024758696556, 0.852094..."
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32...","[1.0590376444160938, 1.0112552838400006, 0.829..."
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3...","[1.0500608645379543, 1.095018446445465, 0.9780..."
...,...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float...","[0.9995186589076184, 1.2212068736553192]"
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.977263942360878, 0.9934241622686386]"
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[0.9572820700705051, 0.9627352058887482]"
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.8675366938114166, 0.8801782131195068]"


In [55]:
def minDistanceFinder(row) : 
    minDistance = min(row["DISTANCES"])
    return minDistance

In [56]:
def minDistanceSentence(row, jobs) : 
    minDistanceIdx = np.argmin(row["DISTANCES"])
    df = jobs[jobs["JOB_ID"] == row["JOB_ID"]]
    if df.empty:
        return ""
    content = df.iloc[0]["JOB_CONTENT"]
    return content[minDistanceIdx]

In [57]:
minDistanceSentence(resultsDf.iloc[6,:], jobsDf)

' You will get the opportunity to grow on the task'

In [58]:
df = jobsDf[jobsDf["JOB_ID"] == 13462067]
df.iloc[0]["JOB_CONTENT"]

['Haas Gartenbau Aggartenner in Garden and landscaping Evzbern Be Languages Number of apprenticeships 1 Swissdoc number 0',
 ' 9 Updated19']

In [59]:
resultsDf["LOWEST_DISTANCE"] = resultsDf.apply(minDistanceFinder, axis =1)
resultsDf["LOWEST_DISTANCE_SENTENCE"] = resultsDf.apply(minDistanceSentence, args=[jobsDf], axis =1)


In [60]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896...",0.670784,You want to gain experience in configuration ...
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960...",0.730214,Further information and application We are at...
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3...","[1.02582386136055, 1.048024758696556, 0.852094...",0.852094,Your profile You have a completed studies as ...
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32...","[1.0590376444160938, 1.0112552838400006, 0.829...",0.814106,Haldi Design AG Schönried
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3...","[1.0500608645379543, 1.095018446445465, 0.9780...",0.923093,Haldi Design AG Schönried
...,...,...,...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float...","[0.9995186589076184, 1.2212068736553192]",0.999519,Haas Gartenbau Aggartenner in Garden and lands...
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.977263942360878, 0.9934241622686386]",0.977264,Haas Gartenbau Aggartenner in Garden and lands...
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[0.9572820700705051, 0.9627352058887482]",0.957282,Haas Gartenbau Aggartenner in Garden and lands...
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.8675366938114166, 0.8801782131195068]",0.867537,Haas Gartenbau Aggartenner in Garden and lands...


In [55]:
#resultsDf.loc[14851,:]

In [56]:
#resultsDf["TOP_ACTIVITY"] = resultsDf.apply(topActivityFinder, axis = 1)

In [57]:
#for id in jobsDf["JOB_ID"] : 
   # print (id)
   # break 

In [61]:
for id in jobsDf["JOB_ID"] :
    job1 = resultsDf[resultsDf["JOB_ID"] == id]
    minEntry = job1["LOWEST_DISTANCE"].idxmin()
    rowMatch = job1.loc[minEntry,:]
    content = slimmedActsDf[slimmedActsDf["ACTIVITY_ID"] == rowMatch["ACTIVITY_ID"]]["ACTIVITY_CONTENT"]
    print(id, minEntry, rowMatch["ACTIVITY_ID"], content)

12220025 194 4.A.4.b.1.G09 205    Manage engineering or construction activities.
Name: ACTIVITY_CONTENT, dtype: object
10814567 413 4.A.4.b.1.G09 205    Manage engineering or construction activities.
Name: ACTIVITY_CONTENT, dtype: object
12401674 632 4.A.4.b.1.G09 205    Manage engineering or construction activities.
Name: ACTIVITY_CONTENT, dtype: object
12473085 853 207 207     Identifying the educational needs of others, ...
Name: ACTIVITY_CONTENT, dtype: object
12905109 1061 196 196     Getting members of a group to work together t...
Name: ACTIVITY_CONTENT, dtype: object
12523090 1173 4.A.2.b.1.G08 82    Select materials for use in green applications...
Name: ACTIVITY_CONTENT, dtype: object
11449790 1471 4.A.3.b.6.G04 168    Prepare environmental reports or documentation.
Name: ACTIVITY_CONTENT, dtype: object
12848898 1744 4.A.4.b.6.G05 222    Advise others on green engineering or design i...
Name: ACTIVITY_CONTENT, dtype: object
12849192 1772 4.A.1.b.2.G03 22    Inspect heating, v

In [62]:
def bestMatchReturn(row, results, actsDf, match, column_name) :
    bestMatch = results[results["JOB_ID"] == row["JOB_ID"]]
    minEntry = bestMatch[match].idxmin()
    rowMatch = bestMatch.loc[minEntry,:]     
    content = actsDf[actsDf["ACTIVITY_ID"] == rowMatch["ACTIVITY_ID"]]                  
    return content[column_name].iloc[0]            


In [63]:
def averageNSentences(row, n):
    return np.average(np.sort(row["DISTANCES"])[:n])

In [64]:
resultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf.apply(averageNSentences, args=[3], axis=1)

In [65]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE,AVERAGE_DISTANCE_3SENT
0,12220025,0,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896...",0.670784,You want to gain experience in configuration ...,0.764877
1,12220025,1,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960...",0.730214,Further information and application We are at...,0.841840
2,12220025,4.A.1.a.1.G01,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.061349783, shape=(), dtype=float3...","[1.02582386136055, 1.048024758696556, 0.852094...",0.852094,Your profile You have a completed studies as ...,0.910633
3,12220025,4.A.1.a.1.G02,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.04309698, shape=(), dtype=float32...","[1.0590376444160938, 1.0112552838400006, 0.829...",0.814106,Haldi Design AG Schönried,0.850310
4,12220025,4.A.1.a.1.G03,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.035856005, shape=(), dtype=float3...","[1.0500608645379543, 1.095018446445465, 0.9780...",0.923093,Haldi Design AG Schönried,0.944726
...,...,...,...,...,...,...,...,...
21895,13462067,4.A.4.b.6.G08,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0028080945, shape=(), dtype=float...","[0.9995186589076184, 1.2212068736553192]",0.999519,Haas Gartenbau Aggartenner in Garden and lands...,1.110363
21896,13462067,226,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.977263942360878, 0.9934241622686386]",0.977264,Haas Gartenbau Aggartenner in Garden and lands...,0.985344
21897,13462067,4.A.4.c.1.G01,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[0.9572820700705051, 0.9627352058887482]",0.957282,Haas Gartenbau Aggartenner in Garden and lands...,0.960009
21898,13462067,228,"((tf.Tensor(-0.002503812, shape=(), dtype=floa...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.8675366938114166, 0.8801782131195068]",0.867537,Haas Gartenbau Aggartenner in Garden and lands...,0.873857


In [66]:
#def activityMatcher(row) :
 #   for id in jobsDf["JOB_ID"] :
  #      bestOverallActivityMatch = np.argmin(resultsDf[row["JOB_ID"] == id]["LOWEST_DISTANCE"])
  #  return bestOverallActivityMatch 
    

In [67]:
column_names = ["JOB_ID", "ISCO", "JOB_CONTENT", "LOWEST_DISTANCE", "BEST_ACTIVITY_MATCH", "BEST_ACTIVITY_ID", "TOP_ACTIVITY_MATCHES", "TOP ACTIVITY_IDS"]

In [68]:
topResultsDf = pd.DataFrame(columns = column_names)

In [69]:
topResultsDf["JOB_ID"] = jobsDf["JOB_ID"]

In [70]:
topResultsDf["ISCO"] = jobsDf["ISCO"]

In [71]:
topResultsDf["JOB_CONTENT"] = jobsDf.apply(lambda x: " ".join(x["JOB_CONTENT"]), axis=1)

In [72]:
topResultsDf["LOWEST_DISTANCE"] = resultsDf["LOWEST_DISTANCE"]
topResultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf["AVERAGE_DISTANCE_3SENT"]

In [73]:
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,LOWEST_DISTANCE,BEST_ACTIVITY_MATCH,BEST_ACTIVITY_ID,TOP_ACTIVITY_MATCHES,TOP ACTIVITY_IDS,AVERAGE_DISTANCE_3SENT
0,12220025,21420,We are a successful and independent engineerin...,0.670784,NaN,NaN,NaN,NaN,0.764877
1,10814567,21420,We are a prestigious and successful engineerin...,0.730214,NaN,NaN,NaN,NaN,0.841840
2,12401674,21420,Bill Weyermann Partner AG has been producing e...,0.852094,NaN,NaN,NaN,NaN,0.910633
3,12473085,62100,Administration Communale de Valderuzforstartin...,0.814106,NaN,NaN,NaN,NaN,0.850310
4,12905109,26220;24210,The Straumann Group is experiencing internatio...,0.923093,NaN,NaN,NaN,NaN,0.944726
...,...,...,...,...,...,...,...,...,...
95,13469747,62100,Commune de Lausanne Service of the Parcs Et Do...,0.829706,NaN,NaN,NaN,NaN,0.861201
96,13494906,24210,Quality Assurance Engineer Bern Switzerland Yo...,0.808829,NaN,NaN,NaN,NaN,0.851269
97,13517245,21420;31180,Solution-oriented creative and dynamic We are ...,0.780521,NaN,NaN,NaN,NaN,0.821306
98,13461980,21620,Green Blue Gardener Garden and Landscaping Efl...,0.749663,NaN,NaN,NaN,NaN,0.797206


In [76]:
topResultsDf["BEST_ACTIVITY_MATCH"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "LOWEST_DISTANCE", "ACTIVITY_CONTENT"), axis=1)
topResultsDf["BEST_ACTIVITY_MATCH_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_CONTENT"), axis=1)

In [77]:
topResultsDf["BEST_ACTIVITY_ID"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "LOWEST_DISTANCE", "ACTIVITY_ID"), axis=1)
topResultsDf["BEST_ACTIVITY_ID_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID"), axis=1)
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,LOWEST_DISTANCE,BEST_ACTIVITY_MATCH,BEST_ACTIVITY_ID,TOP_ACTIVITY_MATCHES,TOP ACTIVITY_IDS,AVERAGE_DISTANCE_3SENT,BEST_ACTIVITY_MATCH_3SENT,BEST_ACTIVITY_ID_3SENT
0,12220025,21420,We are a successful and independent engineerin...,0.670784,Manage engineering or construction activities.,4.A.4.b.1.G09,NaN,NaN,0.764877,Manage engineering or construction activities.,4.A.4.b.1.G09
1,10814567,21420,We are a prestigious and successful engineerin...,0.730214,Manage engineering or construction activities.,4.A.4.b.1.G09,NaN,NaN,0.841840,Manage engineering or construction activities.,4.A.4.b.1.G09
2,12401674,21420,Bill Weyermann Partner AG has been producing e...,0.852094,Manage engineering or construction activities.,4.A.4.b.1.G09,NaN,NaN,0.910633,Manage engineering or construction activities.,4.A.4.b.1.G09
3,12473085,62100,Administration Communale de Valderuzforstartin...,0.814106,"Identifying the educational needs of others, ...",207,NaN,NaN,0.850310,"Identifying the educational needs of others, ...",207
4,12905109,26220;24210,The Straumann Group is experiencing internatio...,0.923093,Getting members of a group to work together t...,196,NaN,NaN,0.944726,Developing specific goals and plans to priorit...,117
...,...,...,...,...,...,...,...,...,...,...,...
95,13469747,62100,Commune de Lausanne Service of the Parcs Et Do...,0.829706,Advise others about land management or conserv...,216,NaN,NaN,0.861201,"Identifying the educational needs of others, ...",207
96,13494906,24210,Quality Assurance Engineer Bern Switzerland Yo...,0.808829,Prepare work requests or proposals.,4.A.3.b.6.G06,NaN,NaN,0.851269,Evaluate materials to assess quality or ensure...,4.A.2.a.3.G01
97,13517245,21420;31180,Solution-oriented creative and dynamic We are ...,0.780521,Advise others on green engineering or design i...,4.A.4.b.6.G05,NaN,NaN,0.821306,Advise others on green engineering or design i...,4.A.4.b.6.G05
98,13461980,21620,Green Blue Gardener Garden and Landscaping Efl...,0.749663,Design green agricultural technologies.,4.A.2.b.2.G03,NaN,NaN,0.797206,Operate robotic or automated equipment.,4.A.3.a.3.G04


In [229]:
print(topResultsDf.iloc[4]["JOB_CONTENT"])
print(topResultsDf.iloc[4]["BEST_ACTIVITY_MATCH"])
print(topResultsDf.iloc[4]["BEST_ACTIVITY_MATCH_3SENT"])

The Straumann Group is experiencing international success in the field of implantology and dental reconstruction as well as tissue regeneration  For more than 60 years we have been focusing our skills on the research and development of optimal solutions for our customers and patients  Implantable medical devices produced on our Villeret site are certified according to MDD 9342ECC ISO 13485 ISO 14001 and FDACGMP 21 CFR 820  In order to ensure that all valid quality documents for the Villeret site are correctly defined in the relevant departments they are Update and made available to users we complete our team with a quality documentation specialist  Your function as part of the management of our Quality System You are part of a team responsible for document management process management and training  In this function you evaluate and process requests for changes attend the departments responsible for the definition and development of the various quality documents and propose continuous 

In [230]:
topResultsDf.to_csv("PrelimResults.csv")